# 导入keras包
keras以TensorFlow作为后端
首先要安装Tensorflow作为后端，可以参考我的安装anaconda教程，通过anaconda安装TensorFlow,创建新环境后在新环境中
```
conda install tensorflow-gpu #需要Nvidia显卡，并安装好显卡驱动 无显卡可conda install tensorflow
```
接着安装keras 
```
conda install keras
```
可以顺带安装一下jupyter notebook （或者不安装直接写py脚本 直接运行）
```
conda install jupyter notebook
```
启动jupyter notebook
```
jupyter notebook
```
然后在jupyter notebook里新建一个python3 notebook可以愉快的在浏览器写脚本啦

我们这里要解决的问题是，将手写数字的灰度图像(28 像素×28 像素)划分到 10 个类别 中(0~9)。我们将使用 MNIST 数据集，它是机器学习领域的一个经典数据集，其历史几乎和这 个领域一样长，而且已被人们深入研究。
这个数据集包含 60 000 张训练图像和 10 000 张测试图 像，由美国国家标准与技术研究院(National Institute of Standards and Technology，即 MNIST 中 的 NIST)在 20 世纪 80 年代收集得到。你可以将“解决”MNIST 问题看作深度学习的“Hello World”，正是用它来验证你的算法是否按预期运行。当你成为机器学习从业者后，会发现 MNIST 一次又一次地出现在科学论文、博客文章等中。

In [1]:

# 首先加载keras自带的mnist数据集
from keras.datasets import mnist
# 自带的mnist第一次需要下载，返回两个元组对象，每个元组里是两个numpy数组，我们把它拆分成四个numpy数组数据集
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()
# 我们可以看看他们的形状
print(train_images.shape,train_labels.shape,test_images.shape,test_labels.shape)

Using TensorFlow backend.


(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


# 接下来的工作流程
- 首先将训练数据 train_images、train_labels、测试数据test_images,test_labels转化成网络要求的数据格式（由灰度（0，255）转换为[0,1]之间的数据
- 其次设计网络模型 选择优化器、损失函数和监控指标
- 最后使用我们训练好的模型对test_image进行预测
神经网络的核心组件是层(layer)，它是一种数据处理模块，你可以将它看成数据过滤器。 进去一些数据，出来的数据变得更加有用。具体来说，层从输入数据中提取表示——我们期望 这种表示有助于解决手头的问题。大多数深度学习都是将简单的层链接起来，从而实现渐进式 的数据蒸馏(data distillation)。深度学习模型就像是数据处理的筛子，包含一系列越来越精细的 数据过滤器(即层)。

本例中的网络包含 2 个 Dense 层，它们是密集连接(也叫全连接)的神经层。第二层(也 是最后一层)是一个 10 路 softmax 层，它将返回一个由 10 个概率值(总和为 1)组成的数组。 每个概率值表示当前数字图像属于 10 个数字类别中某一个的概率

In [2]:
#首先我们导入keras的神经网络模型包models和层包layer
from keras import models
from keras import layers
#定义一个网络的模型对象firstnet
firstnet=models.Sequential()
#Sequential模型代表我们的模型网络是按层顺序叠加的
#为网络添加第一层
#创建一个Dense（全联接层） 使用激活函数为relu函数，输入n个28*28维的向量（1个2D张量），激活后输出n个512维的向量
firstnet.add(layers.Dense(512,activation='relu',input_shape=(28*28,)))
#第二层 softmax层
firstnet.add(layers.Dense(10,activation='softmax'))

要想训练网络，我们还需要选择编译(compile)步骤的三个参数。
- 损失函数(loss function):网络如何衡量在训练数据上的性能，即网络如何朝着正确的
方向前进。
- 优化器(optimizer):基于训练数据和损失函数来更新网络的机制。
- 在训练和测试过程中需要监控的指标(metric):本例只关心精度，即正确分类的图像所
   占的比例。

In [3]:
#编译阶段 使用rmsprop优化器，交叉熵作为损失函数，监控指标为准确度accuracy
firstnet.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])


# 数据处理 
我们的数据格式与编写的网络不同，网络要求为n个28*28维的向量（2D张量），我们的是3D张量（60000，28，28）需要修改成（60000，28*28）
#并且将[0,255]的灰度范围映射到[0,1],并把数据类型由uint更改为float32
在开始训练之前，我们将对数据进行预处理，将其变换为网络要求的形状，并缩放到所 有值都在 [0, 1] 区间。比如，之前训练图像保存在一个 uint8 类型的数组中，其形状为 (60000, 28, 28)，取值区间为[0, 255]。我们需要将其变换为一个float32数组，其形 状为 (60000, 28 * 28)，取值范围为 0~1。

In [4]:
train_images=train_images.reshape(60000,28*28)
train_images=train_images.astype('float32')/255
test_images=test_images.reshape(10000,28*28)
test_images=test_images.astype('float32')/255

In [5]:
#我们还需要对标签进行分类编码，未来将会对这一步骤进行解释。
from keras.utils import to_categorical
train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)
#现在我们准备开始训练网络，在 Keras 中这一步是通过调用网络的 fit 方法来完成的—— 我们在训练数据上拟合(fit)模型。
firstnet.fit(train_images,train_labels,epochs=5,batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 1s 19us/step - loss: 0.2556 - acc: 0.9259
Epoch 2/5
60000/60000 [==============================] - 1s 10us/step - loss: 0.1028 - acc: 0.9689
Epoch 3/5
60000/60000 [==============================] - 1s 10us/step - loss: 0.0676 - acc: 0.9800
Epoch 4/5
60000/60000 [==============================] - 1s 10us/step - loss: 0.0488 - acc: 0.9851
Epoch 5/5
60000/60000 [==============================] - 1s 10us/step - loss: 0.0368 - acc: 0.9889


训练过程中显示了两个数字:一个是网络在训练数据上的损失(loss)，另一个是网络在 训练数据上的精度(acc)。
我们很快就在训练数据上达到了 0.989(98.9%)的精度。现在我们来检查一下模型在测试 集上的性能。

In [6]:
test_loss,test_acc=firstnet.evaluate(test_images,test_labels)
print('test_loss: ',test_loss,'test_acc: ',test_acc)

10000/10000 [==============================] - 0s 15us/step
test_loss:  0.0647473594592 test_acc:  0.9807


In [ ]:
#测试集精度为 97.8%，比训练集精度低不少。训练精度和测试精度之间的这种差距是过拟 合(overfit)造成的。过拟合是指机器学习模型在新数据上的性能往往比在训练数据上要差，它 是第 3 章的核心主题。
#第一个例子到这里就结束了。你刚刚看到了如何构建和训练一个神经网络，用不到 20 行的 Python 代码对手写数字进行分类
#接下来详细介绍这个例子中的每一个步骤，并讲解其背后 的原理。接下来你将要学到张量(输入网络的数据存储对象)、张量运算(层的组成要素)和梯 6 度下降(可以让网络从训练样本中进行学习)。